In [214]:
# The dataset used in this project was downloaded from http://insideairbnb.com.
# using pandas to read in the dataset
import pandas as pd
df = pd.read_csv('/content/Ash_listing.csv') 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)

In [215]:
#check the dataframe
df.info()
df.index
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3009 entries, 0 to 3008
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            3009 non-null   int64  
 1   listing_url                                   3009 non-null   object 
 2   scrape_id                                     3009 non-null   int64  
 3   last_scraped                                  3009 non-null   object 
 4   source                                        3009 non-null   object 
 5   name                                          3009 non-null   object 
 6   description                                   3004 non-null   object 
 7   neighborhood_overview                         2290 non-null   object 
 8   picture_url                                   3009 non-null   object 
 9   host_id                                       3009 non-null   i

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [ ]:
#Remove the dollar signs in 'price'
df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)

In [217]:
df['price'][:10]

0    120.0
1    100.0
     ...  
8    289.0
9     88.0
Name: price, Length: 10, dtype: float64

In [218]:
#some fields are irrelevant for this project, so a few columns were dropped
df_new = df.drop(['scrape_id', 'last_scraped', 'picture_url', 'host_id', 
         'host_url','host_thumbnail_url', 'host_picture_url', 'source'], axis = 1)

In [219]:
#now that the dataset is ready to use, convert it into a sql table
import sqlite3
conn = sqlite3.connect('/df_new.db')
df_new.to_sql('new', conn, if_exists='replace', index=False)

In [220]:
#execute sql queries-testing if the dataset has been converted successfully
conn.execute("""
select id
from new
limit 5
""").fetchall()

[(108061,), (155305,), (156805,), (156926,), (197263,)]

Question 1: What property type is the most expensive property in this dataset? How much does it cost to rent this type of property on average?

In [221]:
#thought process: calculate the average price listing by property types. Round to 2 decimal places. 
#To save some space, only 15 rows are displayed.
conn.execute("""
  select property_type, round(avg(price),2) as "average_price"
  from new
  group by property_type 
  order by average_price desc
  limit 15
""").fetchall()

[('Room in hotel', 1256.5),
 ('Entire cabin', 684.66),
 ('Room in boutique hotel', 647.61),
 ('Entire villa', 499.25),
 ('Room in bed and breakfast', 490.07),
 ('Entire chalet', 346.86),
 ('Private room in bed and breakfast', 341.85),
 ('Entire vacation home', 307.75),
 ('Treehouse', 273.33),
 ('Entire home', 272.84),
 ('Entire townhouse', 268.27),
 ('Entire loft', 265.13),
 ('Entire condo', 251.08),
 ('Shared room in rental unit', 250.0),
 ('Private room in hostel', 214.67)]

Question 2: what type of property is the most reasonably priced?

In [222]:
conn.execute("""
  select a.property_type
  from (select property_type, round(avg(price),2) as 'average_price'
        from new
        group by property_type
        order by average_price
        limit 1) as a
""").fetchall()

[('Private room in hut',)]

Question 3: I'd like to look for something immediately available for at least 5 nights and has the highest rating. Is it possible? If so, how much does it cost per night?

In [223]:
#A list of availabe listings can be retrieved from the following query. 
conn.execute("""
  select listing_url, host_name, review_scores_rating, price
  from new
  where instant_bookable = 't' and availability_30 >= 5
  and review_scores_rating in (select max(review_scores_rating)
                               from new
                               where instant_bookable = 't' and availability_30 >= 5)
  order by price
""").fetchall()

[('https://www.airbnb.com/rooms/38867480', 'Nicole', 5.0, 45.0),
 ('https://www.airbnb.com/rooms/647811956217699952', 'Alon', 5.0, 55.0),
 ('https://www.airbnb.com/rooms/46837010', 'Sam & Gordy', 5.0, 65.0),
 ('https://www.airbnb.com/rooms/50792460', 'Vernon', 5.0, 65.0),
 ('https://www.airbnb.com/rooms/53197970', 'Nicole', 5.0, 65.0),
 ('https://www.airbnb.com/rooms/631998910601981114', 'Laura', 5.0, 72.0),
 ('https://www.airbnb.com/rooms/24287936', 'David', 5.0, 75.0),
 ('https://www.airbnb.com/rooms/602348590325459976', 'Betsy', 5.0, 75.0),
 ('https://www.airbnb.com/rooms/676759193993863114',
  'Paige & Nathan',
  5.0,
  75.0),
 ('https://www.airbnb.com/rooms/690077496342128940', 'Beth', 5.0, 75.0),
 ('https://www.airbnb.com/rooms/677353796235788057', 'Annie', 5.0, 79.0),
 ('https://www.airbnb.com/rooms/51284679', 'Lynn', 5.0, 80.0),
 ('https://www.airbnb.com/rooms/618615434156747025', 'Joy', 5.0, 81.0),
 ('https://www.airbnb.com/rooms/498553730281519631', 'Sabrina', 5.0, 82.0),
 ('